O RSA (Rivest, Shamir e Adleman) é um dos algoritmos de criptografia mais famosos do mundo, ele consiste no uso de duas chaves, uma privada e uma pública, para fazer a lógica de criptografia. A chave privada é de conhecimento exclusivo do portador e a chave publica é conhecida por todos. Com esse par é possível manter a confidencialidade, autenticidade e integridade da mensagem.

Aqui usaremos algumas bibliotecas para ajudar a implementar o código, primeiro utilizaremos a Crypto para geração de numeros primos acima de 1024 bits, para achar o MDC (GCD em inglês), e para achar o inverso modular. Depois utilizaremos a função math para auxiliar os calculos matematicos.

Para implementar o algoritmo RSA devemos seguir os seguintes passos implementados na função achar_chaves():
1 - Escolha dois numeros P e Q primos e que tem ao menos 1024 bits
2 - Calcule N = P*Q e Z = (P-1)*(Q-1) 
3 - Escolha E menor que N e que não tenha fatores em comum com Z além de 1
4 - Encontre um número D tal que ED - 1 seja exatamente divisivel - ou seja modulo ED mod Z = 1 - por Z 
5 - A chave pública KB+ será (N, E)e a chave privada KB- será (N, D)

In [ ]:
from Crypto.Util.number import getPrime, inverse, GCD
import math

def achar_chaves():
    p = int(getPrime(1024))  
    q = int(getPrime(1024)) 
    while p == q:
        q = getPrime(1024) 
   
    n = int(p * q) 
    z = int((p - 1)*(q - 1)) 

    e = getPrime(1024)
    while GCD(e, z) != 1: 
        e = getPrime(1024)

    d = inverse(e, z) 

    kbpublic = (n, e) 
    kbprivate = (n, d) 

    return f" Esta eh a chave publica (n, e): \n{kbpublic}\n Esta eh a chave privada (n, d): \n{kbprivate}"

print(achar_chaves())

 Esta eh a chave publica (n, e): 
(16969230440489812735911846654971789325083154084345181459633191651564149328582089929638759427346257791943938591680498706879972918400938121752407654591200448355163036940201416721971878374574563156964967989173424480607113104027241657627029283368086628698996391411720063039430690283383881015190309351084843219460974867444635231602220229652169606530656658749298980755289283725891664234231856247572226436587564020298973171956085477771916466458241070187126725217576585960966688854205616385573187344843455379701232440203847496195641008443190332310273611550714361454732398358635696753990013152102732434988003148212468926721367, 101453543176151179406650047691559577535427863728269103696919430722273873667562971829563797690956665263553345203588579738242954300946611147880404567910847964023075190941763288342962792774551735737551083002196826737911543224062065188340905194247565353519290895795104196617975871034930001031170415912353850849507),
 Esta eh a chave privada (n, d): 


Aqui implementamos o OAEP (Optimal Asymmetric Encryption Padding) é um tipo de preenchimento usado em criptografia de chave pública para aumentar a segurança do RSA, ele faz isso adicionando uma camada de aleatoriedade a mensagem a ser criptografada. 

Aqui usaremos duas bibliotecas hashlib que usaremos pra gerar um código de hash, mais especificamente SHA3-256, e a OS para interagir com o sistema operacional.

Na função sha3_256(mensagem) fazemos o hash SHA-3 da mensagem.

Na função mfg1(semente, tamanho, hash_func=hashlib.sha3_256) calculamos a mascara (Masck Generation Function 1) que usamos no OAEP. A partir de uma semente geramos uma mascara de comprimento tamanho. Aplicamos o hash sobre a semente + counter.

Na função oaep_codificar(mensagem: bytes, k: int, label: bytes = b"", hash_func=hashlib.sha3_256) -> bytes, aplicamos o OAEP na mensagem original usando SHA3-256, k é o numero de bytes do bloco RSA, aqui adicionamos aleatoridade e padding segura a mensagem. Geramos um hash do label (label_hash) que geralmente esta vazio, depois criamos um DB = label_hash + padding + 0x01 + mensagem, geramos uma semente aleatória de 32 bytes e usamos o MFG1 para mascarar DB e a semente retorna o ME (mensagem encriptada) final: 0x00 || semente mascarada || DB mascarado. 

Na função oaep_decodificar(ME: bytes, k: int, label: bytes = b"", hash_func=hashlib.sha3_256) -> bytes:, revertemos o processo de codificaçao de OAEP, extraimos a mensagem origianal de ME (mensagem encriptada) e verificamos se o label bate com o hash e localizamos a marca 0x01. Extraimos semente_mascaradas e db_mascarado, desmascara os dois usando MGF1, verificamos label_hash no início de DB, procuramos o byte 0x01 como separador entre o padding e a mensagem, no fim retornamos a mensagem.

In [8]:
import hashlib
import os
from key_gen import achar_chaves


def sha3_256(mensagem):
    return hashlib.sha3_256(mensagem).digest() 

def mgf1(semente, tamanho, hash_func=hashlib.sha3_256):
    hashlen = hash_func().digest_size
    mascara = b""
    for counter in range((tamanho + hashlen - 1) // hashlen):
        C = counter.to_bytes(4, byteorder='big')
        mascara += hash_func(semente + C).digest()
    return mascara[:tamanho]

def oaep_codificar(mensagem: bytes, k: int, label: bytes = b"", hash_func=hashlib.sha3_256) -> bytes: 
    mensagemlen = len(mensagem)
    hashlen = hash_func().digest_size

    if mensagemlen > k -2 * hashlen - 2:
        raise ValueError("Mensagem longa demais")
    
    label_hash = hash_func(label).digest()
    PS = b'\x00' * (k - mensagemlen - 2 * hashlen - 2)
    DB = label_hash + PS + b'\x01' + mensagem
    semente = os.urandom(hashlen)
    mascara_db = mgf1(semente, k - hashlen - 1, hash_func)
    db_mascarado = bytes(a ^ b for a, b in zip(DB, mascara_db)) 
    semente_mascara = mgf1(db_mascarado, hashlen, hash_func)
    semente_mascarada = bytes(a ^ b for a, b in zip(semente, semente_mascara))
    return b'\x00' + semente_mascarada + db_mascarado

def oaep_decodificar(ME: bytes, k: int, label: bytes = b"", hash_func=hashlib.sha3_256) -> bytes:
    hashlen = hash_func().digest_size
    if len(ME) != k or ME[0] != 0x00:
        raise ValueError("Bloco inválido")
    
    semente_mascarada = ME[1:hashlen+1]
    db_mascarado = ME[hashlen+1:]

    semente_mascara = mgf1(db_mascarado, hashlen, hash_func)
    semente = bytes(a ^ b for a,b in zip(semente_mascarada, semente_mascara))
    mascara_db = mgf1(semente, k - hashlen - 1, hash_func)
    DB = bytes(a ^ b for a,b in zip(db_mascarado, mascara_db))

    label_hash = hash_func(label).digest()
    if DB[:hashlen] != label_hash:
        raise ValueError("Label hash inválido")
    
    i = DB.find(b'\x01', hashlen)
    if i == -1:
        raise ValueError("Delimitador 0x01 não encontrado")
    return DB[i+1:]

def executar_oaep(mensagem):
    #Aqui geramos as chaves
    chaves = achar_chaves()
    kbpublic = chaves[0]
    kbprivate = chaves[1]

    n, e = kbpublic
    n_priv, d = kbprivate

    k = (kbpublic[0].bit_length() + 7) // 8

    #Aqui codificamos com o OAEP
    me = oaep_codificar(mensagem, k)

    #Criptografar com o RSA
    m_inteiro = int.from_bytes(me, 'big')
    c_inteiro = pow(m_inteiro, e, n)
    criptograma = c_inteiro.to_bytes(k, 'big')

    #Descriptografar com RSA
    c_inteiro = int.from_bytes(criptograma, 'big')
    m_decod_inteiro = pow(c_inteiro, d, n)
    m_decod = m_decod_inteiro.to_bytes(k, 'big')

    #Aqui decodificamos o OAEP
    mensagem_recuperada = oaep_decodificar(m_decod, k)

    return f'Esta eh a mensagem: \n{mensagem}\nEsta eh a mensagem recuperada: \n{mensagem_recuperada}\nO OAEP foi um sucesso: {mensagem==mensagem_recuperada}'

mensagem = b"Este eh um teste da mensagem secreta kkk" # Mensagem que passará pelo OAEP
print(executar_oaep(mensagem))

Esta eh a mensagem: 
b'Este eh um teste da mensagem secreta kkk'
Esta eh a mensagem recuperada: 
b'Este eh um teste da mensagem secreta kkk'
O OAEP foi um sucesso: True


Aqui fazemos a assinatura do documento, implementa um sistema de assinatura digital com RSA e SHA3-256, que permite gerar uma assinatura digital de uma mensagem usando a chave privada; verificar a assinatura digital com a chave pública, confirmando a autenticidade e integridade da mensagem; salvar em arquivo a mensagem junto com sua assinatura e carregar o arquivo e verificar se a assinatura da mensagem está correta.

Aqui usaremos duas bibliotecas hashlib que usaremos pra gerar um código de hash, mais especificamente SHA3-256, e base64 que serve para codificar e decodificar dados binários em texto (base64), facilitando o armazenamento e envio seguro.
 
A função assinar_mensagem(mensagem, chave_privada), gera a assinatura digital de uma mensagem aplicando SHA3-256 e RSA (mensagem^d mod n) e retorna a assinatura codificada em base64.

A função verificar_assinatura(mensagem, assinatura_b64, chave_publica), verifica se a assinatura corresponde ao hash da mensagem e decifra a assinatura com a chave pública (assinatura^e mod n) e compara ao hash real.

A função salvar_arquivo_assinado(nome_arquivo, mensagem, assinatura_b64), salva a mensagem e a assinatura em um arquivo texto, ambas codificadas em base64.

A função carregar_e_verificar_arquivo(nome_arquivo, chave_publica), lê o arquivo, recupera mensagem e assinatura, e verifica se a assinatura é válida, por fim exibe no console se a assinatura está correta.

In [7]:
import base64
import hashlib
from key_gen import *

kbpublic, kbprivate = achar_chaves() 

def assinar_mensagem(mensagem: bytes, chave_privada: tuple) -> str:
    n, d = chave_privada
    hash_mensagem = hashlib.sha3_256(mensagem).digest()
    hash_int = int.from_bytes(hash_mensagem, 'big')
    assinatura = pow(hash_int, d, n)
    assinatura_bytes = assinatura.to_bytes((n.bit_length() + 7) // 8, 'big')
    return base64.b64encode(assinatura_bytes).decode('utf-8')  


def verificar_assinatura(mensagem: bytes, assinatura_b64: str, chave_publica: tuple) -> bool:
    n, e = chave_publica
    assinatura_bytes = base64.b64decode(assinatura_b64)
    assinatura_int = int.from_bytes(assinatura_bytes, 'big')
    hash_verificado = pow(assinatura_int, e, n)
    hash_calculado = int.from_bytes(hashlib.sha3_256(mensagem).digest(), 'big')
    return hash_verificado == hash_calculado


def salvar_arquivo_assinado(nome_arquivo, mensagem: bytes, assinatura_b64: str):
    with open(nome_arquivo, 'w') as f:
        f.write(base64.b64encode(mensagem).decode('utf-8') + "\n")
        f.write(assinatura_b64)


def carregar_e_verificar_arquivo(nome_arquivo, chave_publica: tuple):
    with open(nome_arquivo, 'r') as f:
        mensagem_b64 = f.readline().strip()
        assinatura_b64 = f.readline().strip()
    mensagem = base64.b64decode(mensagem_b64)
    valido = verificar_assinatura(mensagem, assinatura_b64, chave_publica)
    print("Assinatura válida!" if valido else "Assinatura inválida.")

mensagem = b"Aqui vai a mensagem secreta para assinar"# Mensagem que será assinada
print(f'Assinatura base64: \n{assinar_mensagem(mensagem, kbprivate)}')
carregar_e_verificar_arquivo("mensagem_assinada.sig", kbpublic)

Assinatura base64: 
eGFYo+A38gFIrWxETBOHkm6MnxD9sLWQWCvTkhrdu83XGrc8aTFSXoYDl/xJEmqOXE6/xW5t3V2hhS0QUg7GvZoC8LR8IMkcyX5QJtoVa6bWwuQzVBSZ4lV5nR/K2yXq0W9N4OpYwpafLIFzMaAeApfjfoTeqHbxI/oivimkGlTc5+BprhPA4tZFe4PAv+xqfDRPZP4yeqbPovqwI71Gf8pSIrBPT68TUWVxZA7tQXUjOEbCHIsDUZZi/3M30gFU7BX1kzX6C9CLZKUqGCYey+omqQXzcd7Jo7Vo6hm7FAFR34IPiCzJTP5KKH+r2TodP++Gz8L3XYR3Mkno0j9iOg==
Assinatura inválida.
